In [ ]:
# Here we need to load the data, split into training and validation sets.
# Then we have to preprocess the data (normalization, grayscale etc.)

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
xglorot = False
if xglorot:
    def weight_variable(shape, mean, std, name):
        fan_in = np.prod(shape[:-1])
        initial = np.random.randn(*shape) / np.sqrt(fan_in / 2.)
        return tf.Variable(initial, dtype = tf.float32)
else:
    def weight_variable(shape, mean, sigma, name):
         return tf.Variable(tf.random_normal(shape, mean, sigma), name = name)

def bias_variable(shape, name):
    # create bias variable given 1-D shape of fan_out
    return tf.Variable(tf.constant(0.1, shape = shape), dtype = tf.float32, name = name)
def dropout(layer, keep_prob):
    return tf.nn.dropout(layer, keep_prob)   

# L1 through Lout define the depth of each of the layers.
# Named variables and placeholders so the names can be used when the saved session is restored. 
# Named certain operations as well for the same reason
#rate = 5*1e-4
mu = 0
sigma = 0.1
# Here we set the layer depths
L1 = 6
L2 = 16
L3 = 120
L4 = 84
Lout = 43
image_depth = X_train.shape[3]
KEEP_PROB1 = 1.0
KEEP_PROB2 = 1.0

x = tf.placeholder(tf.float32, (None, 32, 32, image_depth), name = "x")
y = tf.placeholder(tf.int32, (None), name = "y")
# Rate is defined as another placeholder, which will be fed for feed_dict
rate = tf.placeholder(tf.float32, (1,), name = "rate")

# SOLUTION: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28xL1.
conv1_W = weight_variable((5, 5, image_depth, L1), mu, sigma, "conv1_W")
conv1_b = bias_variable([L1], "conv1_b")
conv1   = tf.add(tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID'), conv1_b, name = "conv1_op")
#print(conv1.shape)
#conv1_layer = tf.get_variable("conv1_layer", dtype=tf.float32, 
#  initializer=conv1, validate_shape = False)

# SOLUTION: Activation.
conv1 = tf.nn.relu(conv1, name = "conv1_relu")

# SOLUTION: Pooling. Input = 28x28xL1. Output = 14x14xL1.
conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

# SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
#conv2_W = tf.Variable(tf.random_uniform(shape=(5, 5, 6, 16), minval=-2.45/np.sqrt(5*5*6*16), maxval=+2.45/np.sqrt(5*5*6*16)))
conv2_W = weight_variable((5, 5, L1, L2), mu, sigma, "conv2_W")
conv2_b = bias_variable([L2], "conv2_b")
conv2   = tf.add(tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID'), conv2_b, name = "conv2_op")
#print(conv2.shape)
#conv2_layer = tf.get_variable("conv2_layer", dtype=tf.float32, 
# initializer=conv2, validate_shape = False)
# SOLUTION: Activation.
conv2 = tf.nn.relu(conv2, name = "conv2_relu")

# SOLUTION: Pooling. Input = 10x10xL2. Output = 5x5xL2.
conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

# SOLUTION: Flatten. Input = 5x5xL2. Output = 5*5*L2.
fc0   = flatten(conv2)
    
# SOLUTION: Layer 3: Fully Connected. Input = 5*5*L2. Output = L3.
#fc1_W = tf.Variable(tf.random_uniform(shape=(400, 120), minval=-2.45/np.sqrt(400*120), maxval=2.45/np.sqrt(400*120)))
fc1_W = weight_variable((5*5*L2, L3), mu, sigma, "fc1_W")
fc1_b = bias_variable([L3], "fc1_b")
fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
# SOLUTION: Activation.
fc1    = tf.nn.relu(fc1)
fc1 = dropout(fc1, KEEP_PROB1)

# SOLUTION: Layer 4: Fully Connected. Input = L3. Output = L4.
#fc2_W  = tf.Variable(tf.random_uniform(shape=(120, 84), minval=-2.45/np.sqrt(400*120), maxval = 2.45/np.sqrt(400*120)))
fc2_W = weight_variable((L3, L4), mu, sigma, "fc2_W")
fc2_b  = bias_variable([L4], "fc2_b")
fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
# SOLUTION: Activation.
fc2    = tf.nn.relu(fc2)
fc2 = dropout(fc2, KEEP_PROB2)

# SOLUTION: Layer 5: Fully Connected. Input = L4. Output = Lout.
fc3_W = weight_variable((L4, Lout), mu, sigma, "fc3_W")
fc3_b  = bias_variable([Lout], "fc3_b")
logits = tf.add(tf.matmul(fc2, fc3_W), fc3_b, name = "logits_op")


one_hot_y = tf.one_hot(y, Lout, name = "one_hot_op")
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits, name = "cross_entropy_op")
loss_operation = tf.reduce_mean(cross_entropy, name = "loss_op")
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1), name = "correct_pred_op")
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "accuracy_op")

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
#saver = tf.train.Saver()

def evaluate(X_data, y_data, BATCH_SIZE, rate):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, rate: rate})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


In [ ]:
def error_function(X_train, y_train, X_valid, y_valid, rate, epochs, BATCH_SIZE):
    with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    num_batches = np.int(num_examples / BATCH_SIZE)
    print("Training...")
    for i in range(epochs):
        avg_train_cost = 0
        avg_valid_cost = 0
        #X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run([training_operation], feed_dict={x: batch_x, y: batch_y, rate: rate})
            train_cost = sess.run(loss_operation, feed_dict={x:batch_x, y:batch_y})            
            avg_train_cost += train_cost/ (num_batches)
            
        valid_cost = sess.run(loss_operation, feed_dict={x:X_valid, y:y_valid})    
        epochs.append(i+1)
        tc.append(avg_train_cost)
        vc.append(valid_cost)
        validation_accuracy = evaluate(X_valid, y_valid, BATCH_SIZE, rate)
        training_accuracy = evaluate(X_train[:(num_examples)], y_train[:(num_examples)], BATCH_SIZE, rate)
        ta.append(training_accuracy)
        va.append(validation_accuracy)
        print("EPOCH {} ...".format(i+1))        
        print("Average Training Cost = ", avg_train_cost)
        print("Validation Cost = ", valid_cost)
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print()
    plt.figure(1), plt.title("Error vs Epochs")
    fig1 = plt.plot(epochs, tc, 'r-', label = "Training Cost")
    plt.plot(epochs, vc, 'b-', label = "Validation Cost")
    plt.legend(loc = "upper right"), plt.show()
        
    plt.figure(2), plt.title("Accuracy vs Epochs, Vacc : {:0.3f}".format(validation_accuracy))
    fig2 = plt.plot(epochs, ta, 'r-', label = "Training Accuracy")
    plt.plot(epochs, va, 'b-', label = "Validation Accuracy")
    plt.legend(loc = "lower right"), plt.show()
    return valid_cost